In [14]:
#openai api key: sk-proj-G83o8KfgDs6AYLdJJEavT3BlbkFJsZNRXkuczYp2mVTUY6t4
#tavily api key: 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
#link to check account balance: https://platform.openai.com/settings/organization/billing/overview

One of the central concepts of LangGraph is state. Each graph execution creates a state that is passed between nodes in the graph as they execute, and each node updates this internal state with its return value after it executes. The way that the graph updates its internal state is defined by either the type of graph chosen or a custom function.

Let's take a look at a simple example of an agent that can search the web using Tavily Search API.

"Tavily is a platform designed to streamline the integration of artificial intelligence into applications, focusing on natural language processing and machine learning. It offers tools and services such as pre-trained AI models, APIs, and SDKs that enable developers to incorporate advanced AI capabilities like chatbots, text analysis, and content generation into their projects with ease. Tavily provides a user-friendly interface and customization options to tailor AI solutions to specific needs, along with analytics and support to optimize performance and manage implementations effectively." [GPT]

In [15]:
import os

os.environ['OPENAI_API_KEY'] = 'sk-proj-G83o8KfgDs6AYLdJJEavT3BlbkFJsZNRXkuczYp2mVTUY6t4'
os.environ['TAVILY_API_KEY'] = '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'

print(os.environ['OPENAI_API_KEY'])
print(os.environ['TAVILY_API_KEY'])

sk-proj-G83o8KfgDs6AYLdJJEavT3BlbkFJsZNRXkuczYp2mVTUY6t4
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


In [16]:
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langgraph.checkpoint import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

In [17]:
# Define the tools for the agent to use
tools = [TavilySearchResults(max_results=1)]
tool_node = ToolNode(tools)

model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0).bind_tools(tools) #cheapest model with best results

In [18]:
# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [19]:
# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="what is the weather in luxembourg city")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'The current weather in Luxembourg City is as follows:\n- Temperature: 18.4°C (65.1°F)\n- Condition: Partly cloudy\n- Wind: 22.0 km/h from the west\n- Humidity: 77%\n- Precipitation: 1.23 mm\n- Visibility: 10.0 km\n- UV Index: 3.0\n\nFor more detailed information, you can visit [Weather API](https://www.weatherapi.com/).'

Now when we pass the same "thread_id", the conversation context is retained via the saved state (i.e. stored list of messages)

In [20]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="what about zurich")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

'For the current weather in Zurich, you can check the extended weather forecast on [World Weather](https://world-weather.info/forecast/switzerland/zurich/june-2024/).'